## Add a new test

predict()

In [2]:

import os
from face_recognition.web_prediction import predict
from time import sleep
import promptlib
from pkg_resources import resource_filename
from face_recognition.api import *
from processor.store_data import *

method = input('Test by:\n (1) Webcam\n(2) video')

if method != '1' and method != '2':
    exit(1)
elif method == '1':
    predict()
elif method == '2':
    prompter = promptlib.Files()
    abs_path_to_video = prompter.file()
    person_name = ''
    if abs_path_to_video.endswith('mp4') or abs_path_to_video.endswith('mov'):  # add it to known videos
        person_name = str(abs_path_to_video).split('.mp4')[0].split('/')[-1]
        # type = input('Type:\n (1) Known\n(2) Unknown')
        # abs_to_move = None
        # if type == '1':
        #     abs_to_move = os.path.abspath('./test/videos/known')
        # elif type == '2':
        #     abs_to_move = os.path.abspath('./test/videos/unknown')
        # else:
        #     print('Wrong command')
        #     exit(3)
        # print(file)

        # os.rename(file, abs_to_move + '/' + person_name + '.mp4')
        # produce_video(person_name + '.mp4')
        # print(file, abs_to_move)


        # known_people = get_known_people_from_dataset()
        # if person_name in known_people:
        #     _ = input('This person exists in DB. Do you want to append images to already existing directory?\n (y/n)')
        #     if _ == 'n':
        #         exit(4)
        #     elif _ == 'y':
        #         dataset_name_location = get_path_form_name(person_name)
        #         print(dataset_name_location)
        #         video_pic_convertor(abs_path_to_video, dataset_name_location)
        #         store_face_encodings_all()
        #     else:
        #         exit(5)
        # else:
        #     os.mkdir("dataset/" + person_name)
        #     new_directory_name = resource_filename(__name__, "dataset/" + person_name)
        #     dataset_name_location = os.path.abspath(new_directory_name)
        #     video_pic_convertor(abs_path_to_video, dataset_name_location)
        #     sleep(1)
        #     store_face_encodings_all()
        #     pass
    pass


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



524
524
592
592
592
592
592
592
592
592
592
592
592
592
592
592
592
592
592
592
592
592
592
592
592
592
592
592
592
592
592
592
592
592
592
592
592
592
592
592
592
592
592
592
592
592
592
592
592
592
592
592
592
592
592
592
592
592
592
592
592
592
592
592
592
592
592
592
592
592
592
592
592
592
592
495
495
524
592
592
592
592
524
592
592
